In [44]:
import ipywidgets as ipw
import utils
import datetime
import copy
CONFIG = utils.read_json("config_test.json")
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")

In [ ]:
class ObjectPropertiesWidgets(ipw.VBox):
    def __init__(self, task):
        super().__init__()
        self.properties_widgets_list = []
        self.properties_widgets_dict = {}
        self.task = task
    
    def get_properties_widgets(self):
        self.all_schema_classes = DATA_MODEL["classes"]
        self.all_schema_slots = DATA_MODEL["slots"]
        object = self.all_schema_classes[self.task]
        object = self.get_properties_recursive(object)
        for property in object["slots"]:
            if self.all_schema_slots[property]["annotations"]["openbis_type"] != "Not used":
                property_widget, property_dict = self.get_property_widget(property)
                if property_widget:
                    self.properties_widgets_list.append(property_widget)
                    self.properties_widgets_dict[property] = property_dict[property]
                
        self.children = self.properties_widgets_list
    
    def get_property_widget(self, property, is_group = False):
        property_widget = None
        property_dict = None
        property_settings = self.all_schema_slots[property]
        property_multivalued = property_settings["multivalued"]
        property_description = property_settings["description"]
        property_range = property_settings["range"]
        property_openbis_type = property_settings["annotations"]["openbis_type"]
        
        # Multivalued properties are a special case of properties
        if property_multivalued:
            return None, None
        
        if property_openbis_type == "VARCHAR":
            if is_group:
                label_widget = ipw.HTML(value = property_description)
            else:
                label_widget = ipw.HTML(value = f"<b>{property_description}</b>")
                
            text_widget = utils.Text(
                layout = ipw.Layout(width = "200px"), 
                placeholder = "",
            )
            property_widget = ipw.VBox([label_widget, text_widget])
            property_dict = {property: text_widget}
        
        elif property_openbis_type == "MULTILINE_VARCHAR":
            if is_group:
                label_widget = ipw.HTML(value = property_description)
            else:
                label_widget = ipw.HTML(value = f"<b>{property_description}</b>")
                
            textarea_widget = utils.Textarea(
                layout = ipw.Layout(width = "200px", height = "100px"), 
                placeholder = "",
            )
            property_widget = ipw.VBox([label_widget, textarea_widget])
            property_dict = {property: textarea_widget}
        
        elif property_openbis_type == "BOOLEAN":
            if is_group:
                label_widget = ipw.HTML(value = property_description)
            else:
                label_widget = ipw.HTML(value = f"<b>{property_description}</b>")
                
            boolean_widget = utils.Checkbox(
                layout = ipw.Layout(width = "200px"),
                value = False,
                indent = False
            )
            property_widget = ipw.VBox([label_widget, boolean_widget])
            property_dict = {property: boolean_widget}
        
        elif property_openbis_type == "DATE":
            if is_group:
                label_widget = ipw.HTML(value = property_description)
            else:
                label_widget = ipw.HTML(value = f"<b>{property_description}</b>")
                
            datepicker_widget = ipw.DatePicker(
                layout = ipw.Layout(width = "200px"), 
                value = datetime.date.today()
            )
            property_widget = ipw.VBox([label_widget, datepicker_widget])
            property_dict = {property: datepicker_widget}
        
        elif property_openbis_type == "TIMESTAMP":
            if is_group:
                label_widget = ipw.HTML(value = property_description)
            else:
                label_widget = ipw.HTML(value = f"<b>{property_description}</b>")
                
            text_widget = utils.Text(
                layout = ipw.Layout(width = "200px"), 
                placeholder = ""
            )
            property_widget = ipw.VBox([label_widget, text_widget])
            property_dict = {property: text_widget}
        
        elif property_openbis_type == "INTEGER":
            if is_group:
                label_widget = ipw.HTML(value = property_description)
            else:
                label_widget = ipw.HTML(value = f"<b>{property_description}</b>")
                
            int_widget = utils.IntText(
                layout = ipw.Layout(width = "200px")
            )
            property_widget = ipw.VBox([label_widget, int_widget])
            property_dict = {property: int_widget}
        
        elif property_openbis_type == "REAL":
            if is_group:
                label_widget = ipw.HTML(value = property_description)
            else:
                label_widget = ipw.HTML(value = f"<b>{property_description}</b>")
                
            float_widget = utils.FloatText(
                layout = ipw.Layout(width = "200px")
            )
            property_widget = ipw.VBox([label_widget, float_widget])
            property_dict = {property: float_widget}

        elif property_openbis_type == "CONTROLLEDVOCABULARY":
            if is_group:
                label_widget = ipw.HTML(value = property_description)
            else:
                label_widget = ipw.HTML(value = f"<b>{property_description}</b>")
                
            property_vocabulary = DATA_MODEL["enums"][property_range]["permissible_values"].keys()
            property_vocabulary = list(property_vocabulary)
            dropdown_widget = utils.Dropdown(
                layout = ipw.Layout(width = "200px"), 
                options = property_vocabulary,
                value = property_vocabulary[0]
            )
            property_widget = ipw.VBox([label_widget, dropdown_widget])
            property_dict = {property: dropdown_widget}
        
        elif property_openbis_type == "JSON":
            property_widget, property_dict = self.get_property_widgets_recursive(property, property_range)
        
        return property_widget, property_dict

    def get_property_widgets_recursive(self, property, property_range):
        widget_accordion_children = []
        property_dict = {property: {}}
        for slot in self.all_schema_classes[property_range]["slots"]:
            if self.all_schema_slots[slot]["annotations"]["openbis_type"] != "Not used":
                sub_property_widget, sub_property_dict = self.get_property_widget(slot, True)
                property_dict[property][slot] = sub_property_dict[slot]
                widget_accordion_children.append(sub_property_widget)
        property_widget = ipw.HBox(children = widget_accordion_children)
        property_description = self.all_schema_slots[property]["description"]
        
        return ipw.VBox([ipw.HTML(value = f"<b>{property_description}</b>"), property_widget]), property_dict
    
    def get_properties_recursive(self, object):
        # Get all the property types until the last parent class
        object = copy.deepcopy(object)
        object_copy = object
        while "is_a" in object_copy:
            parent_object = self.all_schema_classes[object_copy["is_a"]]
            # Used when the object inherits all the properties from another object
            if object["slots"]:
                object["slots"] = parent_object["slots"] + object["slots"]
            else:
                object["slots"] = parent_object["slots"]
                
            object_copy = parent_object
        
        return object

object_widget = ObjectPropertiesWidgets("Molecule")
display(object_widget)


ObjectPropertiesWidgets(children=(VBox(children=(HTML(value='<b>Name</b>'), Text(value='', layout=Layout(width…

In [58]:
import json
def get_widget_values(widgets_dict):
    def extract_values(d):
        values = {}
        for key, widget in d.items():
            if isinstance(widget, dict):
                # Recursively handle nested dictionaries
                nested_values = extract_values(widget)
                values[key] = json.dumps(nested_values)
            else:
                values[key] = widget.value
        return values

    return extract_values(widgets_dict)

values = get_widget_values(object_widget.properties_widgets_dict)
print(values)

{'name': '', 'empa_number': 0, 'batch': '', 'vial': '', 'hazardous': False, 'hazardous_specification': '', 'dry': False, 'fridge': False, 'no_light': False, 'no_oxygen': False, 'other_storage_condition': False, 'other_storage_condition_specification': '', 'amount': '{"has_value": 0.0, "has_mass_unit": "mg"}', 'chemist_molecule_name': '', 'receive_date': datetime.date(2025, 2, 5), 'comments': ''}
